# 🎯 Multiple Linear Regression: Advanced Sales Analytics

## 🎯 Learning Objectives

In this notebook, you'll master:
- **Multiple Linear Regression**: Using multiple features to predict sales
- **Feature Engineering**: Creating meaningful variables from raw data
- **Multicollinearity Detection**: Identifying and handling correlated features
- **Model Interpretation**: Understanding the business impact of each factor
- **Advanced Diagnostics**: Comprehensive model validation

## 🏢 Business Context: Comprehensive Sales Analysis

As Amazon's senior data scientist, you need to understand:
- **Multiple Factors**: How marketing, traffic, pricing, and promotions work together
- **Interaction Effects**: How different factors amplify or reduce each other's impact
- **Feature Importance**: Which factors drive the most sales growth
- **Optimization Opportunities**: Where to invest for maximum ROI

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
import statsmodels.api as sm
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Set random seed
np.random.seed(42)

print("✅ Libraries imported successfully!")

## 📊 Enhanced Amazon Sales Dataset

Let's create a more comprehensive dataset with additional features:

In [ ]:
# Generate enhanced sales data with more features
np.random.seed(42)
n_samples = 2000

# Base features
marketing_spend = np.random.uniform(10000, 100000, n_samples)
website_traffic = np.random.uniform(50000, 200000, n_samples)
avg_product_price = np.random.uniform(20, 200, n_samples)
seasonal_factor = np.sin(2 * np.pi * np.arange(n_samples) / 365) * 0.3
promotion_active = np.random.choice([0, 1], n_samples, p=[0.7, 0.3])

# Additional features
social_media_presence = np.random.uniform(0, 100, n_samples)  # Social media engagement
customer_reviews = np.random.uniform(3.5, 5.0, n_samples)     # Average product rating
competitor_price = avg_product_price + np.random.normal(0, 10, n_samples)  # Competitor pricing
inventory_level = np.random.uniform(0.3, 1.0, n_samples)      # Stock availability (0-1)
weekend_effect = np.random.choice([0, 1], n_samples, p=[0.7, 0.3])  # Weekend sales boost

# Generate revenue with more complex relationships
revenue = (
    50000 +                    # Base revenue
    0.8 * marketing_spend +    # Marketing impact
    0.3 * website_traffic +    # Traffic impact
    100 * avg_product_price +  # Price impact
    20000 * seasonal_factor +  # Seasonal impact
    15000 * promotion_active + # Promotion impact
    200 * social_media_presence +  # Social media impact
    5000 * customer_reviews +      # Review impact
    -50 * (avg_product_price - competitor_price) +  # Price competition
    10000 * inventory_level +      # Inventory impact
    5000 * weekend_effect +        # Weekend effect
    # Interaction effects
    0.001 * marketing_spend * website_traffic +  # Marketing-Traffic interaction
    -0.1 * avg_product_price * promotion_active +  # Price-Promotion interaction
    np.random.normal(0, 3000, n_samples)  # Random noise
)

# Create comprehensive DataFrame
sales_df = pd.DataFrame({
    'marketing_spend': marketing_spend,
    'website_traffic': website_traffic,
    'avg_product_price': avg_product_price,
    'seasonal_factor': seasonal_factor,
    'promotion_active': promotion_active,
    'social_media_presence': social_media_presence,
    'customer_reviews': customer_reviews,
    'competitor_price': competitor_price,
    'inventory_level': inventory_level,
    'weekend_effect': weekend_effect,
    'revenue': revenue
})

print("📊 Enhanced Amazon Sales Dataset:")
print(sales_df.head())
print(f"\n📈 Dataset Shape: {sales_df.shape}")
print(f"💰 Revenue Range: ${sales_df['revenue'].min():,.0f} - ${sales_df['revenue'].max():,.0f}")

## 🔧 Feature Engineering

Let's create additional meaningful features that could improve our model:

In [ ]:
# Feature Engineering
print("🔧 Creating Engineered Features...")

# 1. Price competitiveness
sales_df['price_competitiveness'] = sales_df['competitor_price'] - sales_df['avg_product_price']

# 2. Marketing efficiency (revenue per marketing dollar)
sales_df['marketing_efficiency'] = sales_df['revenue'] / sales_df['marketing_spend']

# 3. Traffic conversion rate (estimated)
sales_df['estimated_conversion_rate'] = (sales_df['revenue'] / sales_df['avg_product_price']) / sales_df['website_traffic']

# 4. Price-to-quality ratio
sales_df['price_quality_ratio'] = sales_df['avg_product_price'] / sales_df['customer_reviews']

# 5. Marketing-to-traffic ratio
sales_df['marketing_traffic_ratio'] = sales_df['marketing_spend'] / sales_df['website_traffic']

# 6. Inventory efficiency
sales_df['inventory_efficiency'] = sales_df['revenue'] * sales_df['inventory_level']

# 7. Social media ROI
sales_df['social_media_roi'] = sales_df['revenue'] / (sales_df['social_media_presence'] + 1)

# 8. Seasonal marketing effectiveness
sales_df['seasonal_marketing_effect'] = sales_df['marketing_spend'] * sales_df['seasonal_factor']

print("✅ Engineered Features Created:")
engineered_features = [
    'price_competitiveness', 'marketing_efficiency', 'estimated_conversion_rate',
    'price_quality_ratio', 'marketing_traffic_ratio', 'inventory_efficiency',
    'social_media_roi', 'seasonal_marketing_effect'
]
print(f"• {len(engineered_features)} new features created")
print(f"• Total features: {len(sales_df.columns)} (including target)")

# Show correlation with revenue for new features
new_features_corr = sales_df[engineered_features + ['revenue']].corr()['revenue'].sort_values(ascending=False)
print(f"\n📊 Correlation with Revenue (New Features):")
print(new_features_corr)

## 🔍 Multicollinearity Analysis

Before building our model, let's check for multicollinearity (correlated features):

In [ ]:
# Select features for modeling (exclude target and some engineered features to avoid perfect correlation)
feature_columns = [
    'marketing_spend', 'website_traffic', 'avg_product_price', 'seasonal_factor',
    'promotion_active', 'social_media_presence', 'customer_reviews', 
    'competitor_price', 'inventory_level', 'weekend_effect',
    'price_competitiveness', 'marketing_traffic_ratio', 'seasonal_marketing_effect'
]

X = sales_df[feature_columns]
y = sales_df['revenue']

# Correlation matrix for multicollinearity check
correlation_matrix = X.corr()

plt.figure(figsize=(14, 12))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0,
            square=True, linewidths=0.5, fmt='.2f')
plt.title('Feature Correlation Matrix (Multicollinearity Check)', fontsize=16, fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# Find highly correlated features
high_corr_pairs = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) > 0.8:
            high_corr_pairs.append((
                correlation_matrix.columns[i], 
                correlation_matrix.columns[j], 
                correlation_matrix.iloc[i, j]
            ))

print("🔍 Multicollinearity Analysis:")
print("=" * 50)
if high_corr_pairs:
    print("⚠️  Highly Correlated Feature Pairs (|r| > 0.8):")
    for pair in high_corr_pairs:
        print(f"  • {pair[0]} ↔ {pair[1]}: r = {pair[2]:.3f}")
else:
    print("✅ No severe multicollinearity detected (|r| < 0.8)")

# VIF (Variance Inflation Factor) calculation
from statsmodels.stats.outliers_influence import variance_inflation_factor

def calculate_vif(X):
    """Calculate VIF for each feature"""
    vif_data = pd.DataFrame()
    vif_data["Feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return vif_data.sort_values('VIF', ascending=False)

vif_results = calculate_vif(X)
print(f"\n📊 Variance Inflation Factor (VIF):")
print(vif_results)

# Identify problematic features (VIF > 10)
high_vif_features = vif_results[vif_results['VIF'] > 10]
if not high_vif_features.empty:
    print(f"\n⚠️  Features with High VIF (> 10):")
    print(high_vif_features)
else:
    print(f"\n✅ No features with problematic VIF (> 10)")

## 🎯 Multiple Linear Regression Model

Now let's build our comprehensive model:

In [ ]:
# Prepare data for modeling
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Scale features (important for interpretation)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build multiple linear regression model
multiple_model = LinearRegression()
multiple_model.fit(X_train_scaled, y_train)

# Make predictions
y_pred_train = multiple_model.predict(X_train_scaled)
y_pred_test = multiple_model.predict(X_test_scaled)

# Model coefficients
coefficients = pd.DataFrame({
    'Feature': feature_columns,
    'Coefficient': multiple_model.coef_,
    'Abs_Coefficient': np.abs(multiple_model.coef_)
}).sort_values('Abs_Coefficient', ascending=False)

print("🎯 Multiple Linear Regression Results:")
print("=" * 50)
print(f"Intercept: ${multiple_model.intercept_:,.2f}")
print(f"\n📊 Feature Importance (by absolute coefficient):")
print(coefficients.round(2))

# Business interpretation
print(f"\n💼 Business Interpretation:")
print(f"• Base Revenue: ${multiple_model.intercept_:,.2f}")
print(f"• Most Important Factor: {coefficients.iloc[0]['Feature']} (${coefficients.iloc[0]['Coefficient']:,.2f})")
print(f"• Least Important Factor: {coefficients.iloc[-1]['Feature']} (${coefficients.iloc[-1]['Coefficient']:,.2f})")

In [ ]:
# Visualize feature importance
plt.figure(figsize=(12, 8))

# Feature importance plot
colors = ['#FF9900' if x > 0 else '#232F3E' for x in coefficients['Coefficient']]
bars = plt.barh(range(len(coefficients)), coefficients['Abs_Coefficient'], color=colors, alpha=0.7)

# Add coefficient values as text
for i, (idx, row) in enumerate(coefficients.iterrows()):
    plt.text(row['Abs_Coefficient'] + 1000, i, f"${row['Coefficient']:,.0f}", 
             va='center', fontweight='bold')

plt.yticks(range(len(coefficients)), coefficients['Feature'])
plt.xlabel('Absolute Coefficient Value ($)')
plt.title('Feature Importance in Sales Prediction Model', fontsize=16, fontweight='bold')
plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

# Detailed coefficient analysis
print("📊 Detailed Coefficient Analysis:")
print("=" * 50)
for _, row in coefficients.head(5).iterrows():
    impact = "Positive" if row['Coefficient'] > 0 else "Negative"
    print(f"• {row['Feature']}: ${row['Coefficient']:,.0f} ({impact} impact)")
    
    # Business interpretation for top features
    if row['Feature'] == 'marketing_spend':
        print(f"  → Every $1 increase in marketing generates ${row['Coefficient']:.2f} in revenue")
    elif row['Feature'] == 'website_traffic':
        print(f"  → Every additional visitor generates ${row['Coefficient']:.2f} in revenue")
    elif row['Feature'] == 'avg_product_price':
        print(f"  → Every $1 price increase generates ${row['Coefficient']:.2f} in revenue")
    print()

## 📊 Model Evaluation & Comparison

Let's compare our multiple regression model with the simple model from the previous notebook:

In [ ]:
# Calculate comprehensive metrics for multiple regression
def calculate_comprehensive_metrics(y_true, y_pred, model_name):
    """Calculate all evaluation metrics"""
    
    # Basic metrics
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    # Adjusted R-squared
    n = len(y_true)
    p = len(feature_columns)  # number of features
    adj_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
    
    # MAPE
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    
    # Business metrics
    avg_revenue = np.mean(y_true)
    error_percentage = (rmse / avg_revenue) * 100
    
    return {
        'model_name': model_name,
        'mse': mse, 'rmse': rmse, 'mae': mae, 
        'mape': mape, 'r2': r2, 'adj_r2': adj_r2,
        'error_percentage': error_percentage
    }

# Calculate metrics for multiple regression
multiple_metrics = calculate_comprehensive_metrics(y_test, y_pred_test, "Multiple Linear Regression")

# Create comparison table
comparison_data = {
    'Metric': ['MSE', 'RMSE', 'MAE', 'MAPE (%)', 'R²', 'Adjusted R²', 'Error %'],
    'Multiple Regression': [
        f"{multiple_metrics['mse']:,.0f}",
        f"{multiple_metrics['rmse']:,.0f}",
        f"{multiple_metrics['mae']:,.0f}",
        f"{multiple_metrics['mape']:.2f}",
        f"{multiple_metrics['r2']:.4f}",
        f"{multiple_metrics['adj_r2']:.4f}",
        f"{multiple_metrics['error_percentage']:.1f}"
    ]
}

comparison_df = pd.DataFrame(comparison_data)
print("📊 Model Performance Comparison:")
print("=" * 50)
print(comparison_df.to_string(index=False))

# Business interpretation
print(f"\n💼 Business Impact Analysis:")
print(f"• Model Accuracy: {multiple_metrics['r2']*100:.1f}% of variance explained")
print(f"• Prediction Error: ${multiple_metrics['rmse']:,.0f} (±{multiple_metrics['error_percentage']:.1f}%)")
print(f"• Adjusted for Complexity: {multiple_metrics['adj_r2']*100:.1f}% (adjusted R²)")

# Improvement analysis
if 'simple_metrics' in globals():
    improvement_r2 = ((multiple_metrics['r2'] - simple_metrics['r2']) / simple_metrics['r2']) * 100
    improvement_rmse = ((simple_metrics['rmse'] - multiple_metrics['rmse']) / simple_metrics['rmse']) * 100
    print(f"\n📈 Improvement over Simple Model:")
    print(f"• R² Improvement: {improvement_r2:.1f}%")
    print(f"• RMSE Reduction: {improvement_rmse:.1f}%")

## 🔍 Advanced Model Diagnostics

Let's perform comprehensive model diagnostics:

In [ ]:
# Advanced diagnostics using statsmodels
X_with_const = sm.add_constant(X_train_scaled)
model_sm = sm.OLS(y_train, X_with_const).fit()

print("🔍 Advanced Model Diagnostics:")
print("=" * 50)
print(model_sm.summary())

# Residual analysis
residuals = y_test - y_pred_test
predicted = y_pred_test

# Create diagnostic plots
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Multiple Linear Regression Diagnostics', fontsize=16, fontweight='bold')

# 1. Residuals vs Predicted
axes[0, 0].scatter(predicted, residuals, alpha=0.6, color='#FF9900')
axes[0, 0].axhline(y=0, color='red', linestyle='--', alpha=0.8)
axes[0, 0].set_xlabel('Predicted Revenue')
axes[0, 0].set_ylabel('Residuals')
axes[0, 0].set_title('Residuals vs Predicted')
axes[0, 0].grid(True, alpha=0.3)

# 2. Q-Q Plot
from scipy.stats import probplot
probplot(residuals, dist="norm", plot=axes[0, 1])
axes[0, 1].set_title('Q-Q Plot (Normality Check)')
axes[0, 1].grid(True, alpha=0.3)

# 3. Residuals Histogram
axes[0, 2].hist(residuals, bins=30, alpha=0.7, color='#232F3E', edgecolor='black')
axes[0, 2].set_xlabel('Residuals')
axes[0, 2].set_ylabel('Frequency')
axes[0, 2].set_title('Residuals Distribution')
axes[0, 2].grid(True, alpha=0.3)

# 4. Predicted vs Actual
axes[1, 0].scatter(y_test, predicted, alpha=0.6, color='#146EB4')
axes[1, 0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 
                'r--', lw=2, label='Perfect Prediction')
axes[1, 0].set_xlabel('Actual Revenue')
axes[1, 0].set_ylabel('Predicted Revenue')
axes[1, 0].set_title('Predicted vs Actual')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# 5. Residuals vs Index
axes[1, 1].plot(range(len(residuals)), residuals, alpha=0.7, color='#FF6B6B')
axes[1, 1].axhline(y=0, color='red', linestyle='--', alpha=0.8)
axes[1, 1].set_xlabel('Observation Index')
axes[1, 1].set_ylabel('Residuals')
axes[1, 1].set_title('Residuals vs Index')
axes[1, 1].grid(True, alpha=0.3)

# 6. Leverage Plot
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import numpy as np

# Calculate leverage
X_scaled = StandardScaler().fit_transform(X_train)
H = X_scaled @ np.linalg.inv(X_scaled.T @ X_scaled) @ X_scaled.T
leverage = np.diag(H)

axes[1, 2].scatter(leverage, residuals[:len(leverage)], alpha=0.6, color='#4ECDC4')
axes[1, 2].set_xlabel('Leverage')
axes[1, 2].set_ylabel('Residuals')
axes[1, 2].set_title('Residuals vs Leverage')
axes[1, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 🎯 Business Insights & Recommendations

### **Key Findings from Multiple Linear Regression:**

1. **Most Important Factors:**
   - {coefficients.iloc[0]['Feature']}: ${coefficients.iloc[0]['Coefficient']:,.0f} impact
   - {coefficients.iloc[1]['Feature']}: ${coefficients.iloc[1]['Coefficient']:,.0f} impact
   - {coefficients.iloc[2]['Feature']}: ${coefficients.iloc[2]['Coefficient']:,.0f} impact

2. **Model Performance:**
   - **Accuracy**: {multiple_metrics['r2']*100:.1f}% of variance explained
   - **Prediction Error**: ±{multiple_metrics['error_percentage']:.1f}% on average
   - **Complexity Adjusted**: {multiple_metrics['adj_r2']*100:.1f}% (adjusted R²)

3. **Business Recommendations:**
   - **Investment Priority**: Focus on {coefficients.iloc[0]['Feature']} for maximum ROI
   - **Resource Allocation**: Optimize {coefficients.iloc[1]['Feature']} and {coefficients.iloc[2]['Feature']}
   - **Risk Management**: Monitor factors with negative coefficients

### **Strategic Implications:**
- **Marketing Strategy**: Allocate budget based on coefficient importance
- **Pricing Strategy**: Consider price elasticity and competitor pricing
- **Inventory Management**: Optimize stock levels based on demand predictions
- **Performance Monitoring**: Track actual vs predicted performance

### **Model Limitations:**
- **Linear Assumption**: May not capture complex non-linear relationships
- **Feature Interactions**: Limited interaction effects in current model
- **External Factors**: Market conditions, economic factors not included
- **Temporal Effects**: Time-based patterns may need special handling

## 🚀 Next Steps: Advanced Techniques

In the next notebook, we'll explore:

### **Advanced Topics:**
1. **Regularization**: Ridge, Lasso, and Elastic Net regression
2. **Cross-Validation**: Robust model validation techniques
3. **Feature Selection**: Automated feature importance and selection
4. **Model Drift**: Detecting when models need retraining
5. **Hyperparameter Tuning**: Optimizing model parameters

### **Business Applications:**
- **Automated Forecasting**: Real-time sales predictions
- **Dynamic Pricing**: Price optimization based on demand
- **Marketing Attribution**: Understanding campaign effectiveness
- **Risk Assessment**: Identifying sales decline indicators

### **Production Considerations:**
- **Model Deployment**: Putting models into production
- **Performance Monitoring**: Tracking model accuracy over time
- **A/B Testing**: Comparing different model versions
- **Scalability**: Handling large-scale data processing

---

**Ready to master advanced regression techniques?** 🎯